In [ ]:
#To run transformers we would need GPU. Hence, we can you `https://colab.research.google.com/` with free T4 Runtime and execute below code.

#Log-in to HuggingFace.co and generate write TOKEN and put it in colab secret

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124

!pip install -q bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0


from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM,  BitsAndBytesConfig
import torch
from google.colab import userdata


hf_token=userdata.get('HF_TOKEN')

login(hf_token, add_to_git_credential=True)

code_qwen = "Qwen/CodeQwen1.5-7B-Chat"

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell me a joke."}
]

quant_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(code_qwen)   #initialize tokenizer
tokenizer.pad_token=tokenizer.eos_token    #add padding

inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to("cuda")    #tokenize message

model = AutoModelForCausalLM.from_pretrained(code_qwen, device_map="auto", quantization_config=quant_config) #initialize llm model

outputs=model.generate(inputs, max_new_tokens=80)   #use tokenized message to generate output

print(tokenizer.decode(outputs[0]))

